In [ ]:
# in this notebook, I will try to fit a model relating the mean behavior of L4, L2/3, SST and VIP cells

In [ ]:
# load the data

In [1]:
import pyute as ut
import autograd.numpy as np
import matplotlib.pyplot as plt
import sklearn
import h5py
import pdb
import scipy.optimize as sop
from mpl_toolkits.mplot3d import Axes3D
import sklearn.discriminant_analysis as skd
import autograd.scipy.special as ssp
from autograd import elementwise_grad as egrad
from autograd import grad
from autograd import jacobian
from autograd import hessian
import size_contrast_analysis as sca
import scipy.stats as sst
import sim_utils
from importlib import reload
reload(sim_utils)
import calnet.utils
import calnet.fitting_spatial_feature_opto
import opto_utils
%matplotlib notebook

/Users/dan/Documents/code/downloads/OASIS/oasis/functions.py:14: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  "just not the slower interior point methods we compared to in the papers.")


In [2]:
nsize,ncontrast = 5,6

In [3]:
npfile = np.load('rs_vm_denoise_200605.npy',allow_pickle=True)[()]#,{'rs':rs,'rs_denoise':rs_denoise},allow_pickle=True)
rs = npfile['rs']
rs_denoise = npfile['rs_denoise']

In [4]:
nsize,ncontrast,ndir = 5,6,8
ori_dirs = [[0,4],[2,6]] #[[0,4],[1,3,5,7],[2,6]]
nT = len(ori_dirs)
nS = len(rs_denoise[0])

def sum_to_1(r):
    R = r.reshape((r.shape[0],-1))
    R = R/np.nansum(R[:,~np.isnan(R.sum(0))],axis=1)[:,np.newaxis]
    return R

def norm_to_mean(r):
    R = r.reshape((r.shape[0],-1))
    R = R/np.nanmean(R[:,~np.isnan(R.sum(0))],axis=1)[:,np.newaxis]
    return R

Rs = [[None,None] for i in range(len(rs))]
Rso = [[[None for iT in range(nT)] for iS in range(nS)] for icelltype in range(len(rs))]
rso = [[[None for iT in range(nT)] for iS in range(nS)] for icelltype in range(len(rs))]

for iR,r in enumerate(rs_denoise):
    print(iR)
    for ialign in range(nS):
        Rs[iR][ialign] = sum_to_1(r[ialign][:,:nsize,:])
#         Rs[iR][ialign] = von_mises_denoise(Rs[iR][ialign].reshape((-1,nsize,ncontrast,ndir)))



for iR,r in enumerate(rs):
    for ialign in range(nS):
        for iori in range(nT):
            Rso[iR][ialign][iori] = np.nanmean(Rs[iR][ialign].reshape((-1,nsize,ncontrast,ndir))[:,:,:,ori_dirs[iori]],-1)
            Rso[iR][ialign][iori] = Rso[iR][ialign][iori].reshape(Rso[iR][ialign][iori].shape[0],-1)

0
1
2
3
4


In [6]:
def set_bound(bd,code,val=0):
    # set bounds to 0 where 0s occur in 'code'
    for iitem in range(len(bd)):
        bd[iitem][code[iitem]] = val

In [7]:
nN = 30
nS = 2
nP = 2
nT = 2
nQ = 4

# code for bounds: 0 , constrained to 0
# +/-1 , constrained to +/-1
# 1.5, constrained to [0,1]
# 2 , constrained to [0,inf)
# -2 , constrained to (-inf,0]
# 3 , unconstrained

Wmx_bounds = 3*np.ones((nP,nQ),dtype=int)
Wmx_bounds[0,1] = 0 # SSTs don't receive L4 input

Wsx_bounds = 3*np.ones(Wmx_bounds.shape) #Wmx_bounds.copy()*0 #np.zeros_like(Wmx_bounds)
Wsx_bounds[0,1] = 0

Wmy_bounds = 3*np.ones((nQ,nQ),dtype=int)
Wmy_bounds[0,:] = 2 # PCs are excitatory
Wmy_bounds[1:,:] = -2 # all the cell types except PCs are inhibitory
Wmy_bounds[1,1] = 0 # SSTs don't inhibit themselves
# Wmy_bounds[3,1] = 0 # PVs are allowed to inhibit SSTs, consistent with Hillel's unpublished results, but not consistent with Pfeffer et al.
Wmy_bounds[2,0] = 0 # VIPs don't inhibit L2/3 PCs. According to Pfeffer et al., only L5 PCs were found to get VIP inhibition

Wsy_bounds = 3*np.ones(Wmy_bounds.shape) #Wmy_bounds.copy()*0 #np.zeros_like(Wmy_bounds)
Wsy_bounds[1,1] = 0
Wsy_bounds[3,1] = 0 

def tile_nS_nT_nN(kernel):
    row = np.concatenate([kernel for idim in range(nS*nT)],axis=0)[np.newaxis,:]
    tiled = np.concatenate([row for irow in range(nN)],axis=0)
    return tiled

s02_bounds = np.ones((nQ,))

k_bounds = 1.5*np.ones((nQ,))

kappa_bounds = np.ones((1,))
# kappa_bounds = 2*np.ones((1,))

T_bounds = 1.5*np.ones((nQ,))

X_bounds = tile_nS_nT_nN(np.array([2,1]))
# X_bounds = np.array([np.array([2,1,2,1])]*nN)

Xp_bounds = tile_nS_nT_nN(np.array([3,1]))
# Xp_bounds = np.array([np.array([3,1,3,1])]*nN)

# Y_bounds = tile_nS_nT_nN(2*np.ones((nQ,)))
# # Y_bounds = 2*np.ones((nN,nT*nS*nQ))

Eta_bounds = tile_nS_nT_nN(3*np.ones((nQ,)))
# Eta_bounds = 3*np.ones((nN,nT*nS*nQ))

Xi_bounds = tile_nS_nT_nN(3*np.ones((nQ,)))
# Xi_bounds = 3*np.ones((nN,nT*nS*nQ))

h_bounds = -2*np.ones((1,))


In [8]:
# shapes = [(nP,nQ),(nQ,nQ),(nP,nQ),(nQ,nQ),(nQ,),(nQ,),(1,),(nN,nS*nP),(nN,nS*nQ),(nN,nS*nQ),(nN,nS*nQ)]
shapes = [(nP,nQ),(nQ,nQ),(nP,nQ),(nQ,nQ),(nQ,),(nQ,),(1,),(nQ,),(nN,nT*nS*nP),(nN,nT*nS*nP),(nN,nT*nS*nQ),(nN,nT*nS*nQ),(1,)]
#         Wmx,    Wmy,    Wsx,    Wsy,    s02,  k,    kappa,T,   XX,            XXp,          Eta,          Xi

lb = [-np.inf*np.ones(shp) for shp in shapes]
ub = [np.inf*np.ones(shp) for shp in shapes]
bdlist = [Wmx_bounds,Wmy_bounds,Wsx_bounds,Wsy_bounds,s02_bounds,k_bounds,kappa_bounds,T_bounds,X_bounds,Xp_bounds,Eta_bounds,Xi_bounds,h_bounds]

set_bound(lb,[bd==0 for bd in bdlist],val=0)
set_bound(ub,[bd==0 for bd in bdlist],val=0)

set_bound(lb,[bd==2 for bd in bdlist],val=0)

set_bound(ub,[bd==-2 for bd in bdlist],val=0)

set_bound(lb,[bd==1 for bd in bdlist],val=1)
set_bound(ub,[bd==1 for bd in bdlist],val=1)

set_bound(lb,[bd==1.5 for bd in bdlist],val=0)
set_bound(ub,[bd==1.5 for bd in bdlist],val=1)

set_bound(lb,[bd==-1 for bd in bdlist],val=-1)
set_bound(ub,[bd==-1 for bd in bdlist],val=-1)

# for bd in [lb,ub]:
#     for ind in [2,3]:
#         bd[ind][:,1] = 0

# temporary for no variation expt.
# lb[2] = np.zeros_like(lb[2])
# lb[3] = np.zeros_like(lb[3])
# lb[4] = np.ones_like(lb[4])
# lb[5] = np.zeros_like(lb[5])
# ub[2] = np.zeros_like(ub[2])
# ub[3] = np.zeros_like(ub[3])
# ub[4] = np.ones_like(ub[4])
# ub[5] = np.ones_like(ub[5])
# temporary for no variation expt.
lb = np.concatenate([a.flatten() for a in lb])
ub = np.concatenate([b.flatten() for b in ub])
bounds = [(a,b) for a,b in zip(lb,ub)]

In [10]:
nS = 2
ndims = 5
ncelltypes = 5
Yhat = [[None for iT in range(nT)] for iS in range(nS)]
Xhat = [[None for iT in range(nT)] for iS in range(nS)]
Ypc_list = [[None for iT in range(nT)] for iS in range(nS)]
Xpc_list = [[None for iT in range(nT)] for iS in range(nS)]
for iS in range(nS):
    mx = np.zeros((ncelltypes,))
    yy = [None for icelltype in range(ncelltypes)]
    for icelltype in range(ncelltypes):
        yy[icelltype] = np.nanmean(Rso[icelltype][iS][0],0)
        mx[icelltype] = np.nanmax(yy[icelltype])
    for iT in range(nT):
        y = [np.nanmean(Rso[icelltype][iS][iT],axis=0)[:,np.newaxis]/mx[icelltype] for icelltype in range(1,ncelltypes)]
        Ypc_list[iS][iT] = [None for icelltype in range(1,ncelltypes)]
        for icelltype in range(1,ncelltypes):
            rss = Rso[icelltype][iS][iT].copy() #.reshape(Rs[icelltype][ialign].shape[0],-1)
    #         print(rss.max())
    #         rss[rss<0] = 0
    #         rss = rss[np.random.randn(rss.shape[0])>0]
            try:
                u,s,v = np.linalg.svd(rss-np.mean(rss,0)[np.newaxis])
                Ypc_list[iS][iT][icelltype-1] = [(s[idim],v[idim]) for idim in range(ndims)]
#                 print('yep on Y')
#                 print(np.min(np.sum(rs[icelltype][iS][iT],axis=1)))
            except:
#                 print('nope on Y')
                print(np.mean(np.isnan(rss)))
                print(np.min(np.sum(rs[icelltype][iS][iT],axis=1)))
        Yhat[iS][iT] = np.concatenate(y,axis=1)
#         x = sim_utils.columnize(Rso[0][iS][iT])[:,np.newaxis]
        icelltype = 0
        x = np.nanmean(Rso[icelltype][iS][iT],0)[:,np.newaxis]/mx[icelltype]
#         opto_column = np.concatenate((np.zeros((nN,)),np.zeros((nNO/2,)),np.ones((nNO/2,))),axis=0)[:,np.newaxis]
        Xhat[iS][iT] = np.concatenate((x,np.ones_like(x)),axis=1)
#         Xhat[iS][iT] = np.concatenate((x,np.ones_like(x),opto_column),axis=1)
        icelltype = 0
        rss = Rso[icelltype][iS][iT].copy()
#         try:
        u,s,v = np.linalg.svd(rss-rss.mean(0)[np.newaxis])
        Xpc_list[iS][iT] = [None for iinput in range(2)]
        Xpc_list[iS][iT][0] = [(s[idim],v[idim]) for idim in range(ndims)]
        Xpc_list[iS][iT][1] = [(0,np.zeros((Xhat[0][0].shape[0],))) for idim in range(ndims)]
#         except:
#             print('nope on X')
#             print(np.mean(np.isnan(rss)))
#             print(np.min(np.sum(Rso[icelltype][iS][iT],axis=1)))
nN,nP = Xhat[0][0].shape
nQ = Yhat[0][0].shape[1]

In [11]:
def compute_f_(Eta,Xi,s02):
    return sim_utils.f_miller_troyer(Eta,Xi**2+np.concatenate([s02 for ipixel in range(nS*nT)]))
def compute_fprime_m_(Eta,Xi,s02):
    return sim_utils.fprime_miller_troyer(Eta,Xi**2+np.concatenate([s02 for ipixel in range(nS*nT)]))*Xi
def compute_fprime_s_(Eta,Xi,s02):
    s2 = Xi**2+np.concatenate((s02,s02),axis=0)
    return sim_utils.fprime_s_miller_troyer(Eta,s2)*(Xi/s2)
def sorted_r_eigs(w):
    drW,prW = np.linalg.eig(w)
    srtinds = np.argsort(drW)
    return drW[srtinds],prW[:,srtinds]

In [12]:
#         0.Wmx,  1.Wmy,  2.Wsx,  3.Wsy,  4.s02,5.K,  6.kappa,7.T,8.XX,        9.XXp,        10.Eta,       11.Xi

shapes = [(nP,nQ),(nQ,nQ),(nP,nQ),(nQ,nQ),(nQ,),(nQ,),(1,),(nQ,),(nN,nT*nS*nP),(nN,nT*nS*nP),(nN,nT*nS*nQ),(nN,nT*nS*nQ),(1,)]


In [13]:
import calnet.fitting_spatial_feature
import sim_utils

In [14]:
opto_dict = np.load('../shared_data/vip_halo_data_for_sim.npy',allow_pickle=True)[()]

In [15]:
Yhat_opto = opto_dict['Yhat_opto']
h_opto = opto_dict['h_opto']
dYY = Yhat_opto[1::2]-Yhat_opto[0::2]
for to_overwrite in [1,2,5,6]:
    dYY[:,to_overwrite] = dYY[:,to_overwrite+8]
for to_overwrite in [11,15]:
    dYY[:,to_overwrite] = dYY[:,to_overwrite-8]

In [22]:
from importlib import reload
reload(calnet)
reload(calnet.fitting_spatial_feature_opto)
reload(sim_utils)
# reload(calnet.fitting_spatial_feature)
# W0list = [np.ones(shp) for shp in shapes]
wt_dict = {}
wt_dict['X'] = 1
wt_dict['Y'] = 1
wt_dict['Eta'] = 1# 10
wt_dict['Xi'] = 1
wt_dict['stims'] = np.ones((30,1)) #(np.arange(30)/30)[:,np.newaxis]**1 #
wt_dict['barrier'] = 0. #30.0 #0.1
wt_dict['opto'] = 1e3
ntries = 1
nhyper = 1
dt = 1e-1
niter = int(np.round(50/dt)) #int(1e4)
perturbation_size = 5e-2
# learning_rate = 1e-4 # 1e-5 #np.linspace(3e-4,1e-3,niter+1) # 1e-5
l2_penalty = 0.1
Wt = [[None for itry in range(ntries)] for ihyper in range(nhyper)]
loss = np.zeros((nhyper,ntries))
is_neg = np.array([b[1] for b in bounds])==0
counter = 0
negatize = [np.zeros(shp,dtype='bool') for shp in shapes]
for ishp,shp in enumerate(shapes):
    nel = np.prod(shp)
    negatize[ishp][:][is_neg[counter:counter+nel].reshape(shp)] = True
    counter = counter + nel
for ihyper in range(nhyper):
    for itry in range(ntries):
        print((ihyper,itry))
        W0list = [0.01*(ihyper+1)*np.random.rand(*shp) for shp in shapes]
        counter = 0
        for ishp,shp in enumerate(shapes):
            W0list[ishp][negatize[ishp]] = -W0list[ishp][negatize[ishp]]
        W0list[5] = np.ones(shapes[5]) # K
        W0list[6] = np.ones(shapes[6]) # kappa
        W0list[7] = np.ones(shapes[7]) # T
        W0list[8] = np.concatenate(Xhat,axis=1) #XX
        W0list[9] = np.zeros_like(W0list[8]) #XXp
        W0list[10] = np.zeros(shapes[10]) #Eta
        W0list[11] = np.zeros(shapes[11]) #Xi
        #[Wmx,Wmy,Wsx,Wsy,s02,k,kappa,T,XX,XXp,Eta,Xi]
#         W0list = Wstar_dict['as_list'].copy()
#         W0list[1] = W0list[1] + 0.05*np.random.randn(*W0list[1].shape)
#         W0list[1][1,0] = -1.5
#         W0list[1][3,0] = -1.5
#         W0list[1][0,0] = 1.5
        # wt_dict['Xi'] = 10
        # wt_dict['Eta'] = 10
        Wt[ihyper][itry],loss[ihyper][itry],gr,hess,result = calnet.fitting_spatial_feature_opto.fit_W_sim(Xhat,Xpc_list,\
            Yhat,Ypc_list,pop_rate_fn=sim_utils.f_miller_troyer,pop_deriv_fn=sim_utils.fprime_miller_troyer,\
            neuron_rate_fn=sim_utils.evaluate_f_mt,W0list=W0list.copy(),bounds=bounds,niter=niter,wt_dict=wt_dict,\
            l2_penalty=l2_penalty,compute_hessian=False,dt=dt,perturbation_size=perturbation_size,dYY=dYY)
#         Wt[ihyper][itry] = [w[-1] for w in Wt_temp]
#         loss[ihyper,itry] = loss_temp[-1]

(0, 0)
X:17.238247
Y:16.285855
Eta:0.379005
Xi:1.511998
Opto:3.505661
cost: 38.920766128115794
cost: Autograd ArrayBox with value 38.92076612811577
X:17.102070
Y:25.337584
Eta:724.261937
Xi:3072.314493
Opto:3.504083
cost: 3842.520167065096
cost: Autograd ArrayBox with value 3842.5201670650836
X:17.235205
Y:16.325495
Eta:0.015802
Xi:0.036863
Opto:3.505631
cost: 37.11899496029323
cost: Autograd ArrayBox with value 37.118994960293264
X:17.231930
Y:16.328128
Eta:0.008652
Xi:0.003957
Opto:3.505624
cost: 37.0782916365041
cost: Autograd ArrayBox with value 37.07829163650405
X:17.227342
Y:16.326615
Eta:0.007414
Xi:0.001750
Opto:3.505618
cost: 37.068739497172594
cost: Autograd ArrayBox with value 37.068739497172594
X:17.217517
Y:16.322908
Eta:0.006323
Xi:0.004733
Opto:3.505605
cost: 37.05708630271226
cost: Autograd ArrayBox with value 37.057086302712285
X:17.185367
Y:16.310921
Eta:0.004716
Xi:0.015778
Opto:3.505563
cost: 37.02234539931482
cost: Autograd ArrayBox with value 37.0223453993149
X:17

cost: Autograd ArrayBox with value 18.502332286590338
X:7.055603
Y:6.288371
Eta:2.093539
Xi:0.412415
Opto:2.500071
cost: 18.349998833908817
cost: Autograd ArrayBox with value 18.349998812831398
X:7.023734
Y:6.241960
Eta:1.802597
Xi:0.412756
Opto:2.481520
cost: 17.962566972999362
cost: Autograd ArrayBox with value 17.9625671647567
X:6.984314
Y:6.249421
Eta:1.466263
Xi:0.435364
Opto:2.473466
cost: 17.60882802475582
cost: Autograd ArrayBox with value 17.608829060576447
X:6.993420
Y:5.805132
Eta:1.783666
Xi:1.342856
Opto:2.485997
cost: 18.411070672679273
cost: Autograd ArrayBox with value 18.411061385799165
X:6.985558
Y:6.157114
Eta:1.468738
Xi:0.460312
Opto:2.474766
cost: 17.546487825481616
cost: Autograd ArrayBox with value 17.54649229140237
X:6.959853
Y:6.267562
Eta:1.271776
Xi:0.466298
Opto:2.481086
cost: 17.44657514849542
cost: Autograd ArrayBox with value 17.446568312385704
X:6.973948
Y:5.972038
Eta:1.498152
Xi:0.544845
Opto:2.485467
cost: 17.4744499530709
cost: Autograd ArrayBox wit

cost: Autograd ArrayBox with value 13.62128326047278
X:7.186901
Y:1.063703
Eta:0.954152
Xi:1.911643
Opto:2.480807
cost: 13.597205705188605
cost: Autograd ArrayBox with value 13.597204685709082
X:7.176623
Y:0.955878
Eta:1.114020
Xi:2.035632
Opto:2.481271
cost: 13.763424342228483
cost: Autograd ArrayBox with value 13.763431408017212
X:7.184248
Y:1.030193
Eta:0.944423
Xi:1.937003
Opto:2.480232
cost: 13.576098772024098
cost: Autograd ArrayBox with value 13.57610973993419
X:7.184071
Y:0.950667
Eta:0.937331
Xi:1.984546
Opto:2.479538
cost: 13.53615311192959
cost: Autograd ArrayBox with value 13.536150042707405
X:7.180804
Y:0.923008
Eta:0.933472
Xi:1.996291
Opto:2.478431
cost: 13.512005789513738
cost: Autograd ArrayBox with value 13.51200611208677
X:7.170408
Y:0.917521
Eta:0.921397
Xi:1.989678
Opto:2.476460
cost: 13.475465050949026
cost: Autograd ArrayBox with value 13.475464691851009
X:7.159936
Y:0.926070
Eta:0.908473
Xi:1.975716
Opto:2.475713
cost: 13.445907506065815
cost: Autograd ArrayBox 

cost: Autograd ArrayBox with value 11.986563522619653
X:6.950427
Y:0.401085
Eta:0.830776
Xi:1.347104
Opto:2.426818
cost: 11.956209041002953
cost: Autograd ArrayBox with value 11.956206007565694
X:6.952538
Y:0.321244
Eta:0.880109
Xi:1.369325
Opto:2.428860
cost: 11.952077039344992
cost: Autograd ArrayBox with value 11.952073793852541
X:6.951347
Y:0.359861
Eta:0.850293
Xi:1.355597
Opto:2.427757
cost: 11.944855119005151
cost: Autograd ArrayBox with value 11.944857852311129
X:6.951561
Y:0.341399
Eta:0.851966
Xi:1.356294
Opto:2.429499
cost: 11.930718437024098
cost: Autograd ArrayBox with value 11.930715574359787
X:6.951827
Y:0.327939
Eta:0.854131
Xi:1.360510
Opto:2.428570
cost: 11.92297537102822
cost: Autograd ArrayBox with value 11.92297249962338
X:6.951979
Y:0.323394
Eta:0.857045
Xi:1.359305
Opto:2.426639
cost: 11.918362098176218
cost: Autograd ArrayBox with value 11.918366697485407
X:6.952104
Y:0.309563
Eta:0.858737
Xi:1.361162
Opto:2.422271
cost: 11.903837077924145
cost: Autograd ArrayBo

cost: Autograd ArrayBox with value 11.000908615639727
X:6.975961
Y:-0.398569
Eta:0.819919
Xi:1.211004
Opto:2.376288
cost: 10.984602092914127
cost: Autograd ArrayBox with value 10.984576697217864
X:6.975372
Y:-0.397093
Eta:0.814863
Xi:1.209934
Opto:2.376282
cost: 10.97935698887736
cost: Autograd ArrayBox with value 10.979296556385156
X:6.974900
Y:-0.401404
Eta:0.811500
Xi:1.209445
Opto:2.376081
cost: 10.970521932653982
cost: Autograd ArrayBox with value 10.970442842800413
X:6.974308
Y:-0.426906
Eta:0.817001
Xi:1.211045
Opto:2.375336
cost: 10.950783529970977
cost: Autograd ArrayBox with value 10.95062832633743
X:6.974578
Y:-0.447413
Eta:0.814614
Xi:1.209613
Opto:2.374324
cost: 10.92571713882892
cost: Autograd ArrayBox with value 10.925720144498232
X:6.980359
Y:-0.607673
Eta:0.868385
Xi:1.286340
Opto:2.367543
cost: 10.894954013117328
cost: Autograd ArrayBox with value 10.895137120523588
X:6.976776
Y:-0.532419
Eta:0.819237
Xi:1.221669
Opto:2.370397
cost: 10.855659424899306
cost: Autograd A

X:7.051832
Y:-1.603113
Eta:0.903599
Xi:1.249068
Opto:2.330698
cost: 9.932083860914378
cost: Autograd ArrayBox with value 9.931168151852408
X:7.065525
Y:-1.611478
Eta:0.889664
Xi:1.248814
Opto:2.326536
cost: 9.919061153904117
cost: Autograd ArrayBox with value 9.921367151735826
X:7.058437
Y:-1.606341
Eta:0.891517
Xi:1.237718
Opto:2.328495
cost: 9.909826332320574
cost: Autograd ArrayBox with value 9.910526239507668
X:7.061210
Y:-1.606796
Eta:0.883120
Xi:1.216737
Opto:2.328175
cost: 9.882445392805948
cost: Autograd ArrayBox with value 9.882367237004233
X:7.060240
Y:-1.575027
Eta:0.887169
Xi:1.164828
Opto:2.329093
cost: 9.8663028661277
cost: Autograd ArrayBox with value 9.867186861904626
X:7.061362
Y:-1.583400
Eta:0.886549
Xi:1.167147
Opto:2.328874
cost: 9.860531037844789
cost: Autograd ArrayBox with value 9.860006916273337
X:7.064737
Y:-1.615878
Eta:0.894778
Xi:1.171240
Opto:2.328242
cost: 9.843119084630722
cost: Autograd ArrayBox with value 9.842245413661553
X:7.066032
Y:-1.632067
Eta:0.

cost: Autograd ArrayBox with value 9.095957946599611
X:7.113458
Y:-1.592846
Eta:0.601985
Xi:0.652670
Opto:2.306534
cost: 9.081801297862441
cost: Autograd ArrayBox with value 9.081872734273128
X:7.115835
Y:-1.661930
Eta:0.674311
Xi:0.629179
Opto:2.304976
cost: 9.062371412936498
cost: Autograd ArrayBox with value 9.063223732571728
X:7.116434
Y:-1.640402
Eta:0.615104
Xi:0.630829
Opto:2.303689
cost: 9.025654737977366
cost: Autograd ArrayBox with value 9.025260967704813
X:7.116807
Y:-1.657289
Eta:0.601764
Xi:0.640688
Opto:2.303779
cost: 9.005748877234726
cost: Autograd ArrayBox with value 9.00554004540382
X:7.117258
Y:-1.697501
Eta:0.603393
Xi:0.657585
Opto:2.305932
cost: 8.986667346106142
cost: Autograd ArrayBox with value 8.986400048934918
X:7.117615
Y:-1.719965
Eta:0.599378
Xi:0.666311
Opto:2.308009
cost: 8.971348369190071
cost: Autograd ArrayBox with value 8.972211846556727
X:7.119416
Y:-1.781188
Eta:0.611216
Xi:0.710955
Opto:2.312763
cost: 8.973162092252444
cost: Autograd ArrayBox with

In [285]:
itry = 0
Wmx,Wmy,Wsx,Wsy,s02,K,kappa,T,XX,XXp,Eta,Xi,h = parse_W(Wt[0][itry])

In [286]:
def parse_W(W):
    Wmx,Wmy,Wsx,Wsy,s02,K,kappa,T,XX,XXp,Eta,Xi,h = W
    return Wmx,Wmy,Wsx,Wsy,s02,K,kappa,T,XX,XXp,Eta,Xi,h

In [ ]:
np.save(weights_file,allow_pickle=True)[()]